1. 문서의 내용을 읽는다
2. 문서를 쪼갠다
    - 토큰수 초과로 답변을 생성하지 못할 수 있고
    - 문서가 길면 (인풋이 길면) 답변 생성이 오래걸림
3. 임베딩 -> Vector Database에 저장
4. 질문이 있을 때, Vector Database에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

# 1. 패키지 설치

In [1]:
%pip install python-dotenv langchain langchain-openai langchain-community langchain-text-splitters docx2txt langchain-chroma


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# 2. Knowledge Base 구성을 위한 데이터 생성

- [RecursiveCharacterTextSplitter](https://python.langchain.com/v0.2/docs/how_to/recursive_text_splitter/)를 활용한 데이터 chunking
    - split 된 데이터 chunk를 Large Language Model(LLM)에게 전달하면 토큰 절약 가능
    - 비용 감소와 답변 생성시간 감소의 효과
    - LangChain에서 다양한 [TextSplitter](https://python.langchain.com/v0.2/docs/how_to/#text-splitters)들을 제공
- `chunk_size` 는 split 된 chunk의 최대 크기
- `chunk_overlap`은 앞 뒤로 나뉘어진 chunk들이 얼마나 겹쳐도 되는지 지정

In [2]:
# 문서 쪼개기
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader('./tax_docs/tax_with_markdown.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [3]:
# 쪼갠결과
print(type(document_list))
print(len(document_list))
document_list[:3]

<class 'list'>
225


[Document(metadata={'source': './tax_docs/tax_with_markdown.docx'}, page_content='소득세법\n\n소득세법\n\n[시행 2024. 1. 1.] [법률 제19933호, 2023. 12. 31., 일부개정]\n\n기획재정부(소득세제과(사업소득, 기타소득)) 044-215-4217\n\n기획재정부(소득세제과(근로소득)) 044-215-4216\n\n기획재정부(재산세제과(양도소득세)) 044-215-4314\n\n기획재정부(금융세제과(이자소득, 배당소득)) 044-215-4236\n\n\n\n\t제1장 총칙 <개정 2009. 12. 31.>\t\n\n\n\n제1조(목적) 이 법은 개인의 소득에 대하여 소득의 성격과 납세자의 부담능력 등에 따라 적정하게 과세함으로써 조세부담의 형평을 도모하고 재정수입의 원활한 조달에 이바지함을 목적으로 한다.\n\n[본조신설 2009. 12. 31.]\n\n[종전 제1조는 제2조로 이동 <2009. 12. 31.>]\n\n\n\n제1조의2(정의) ① 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2010. 12. 27., 2014. 12. 23., 2018. 12. 31.>\n\n1. “거주자”란 국내에 주소를 두거나 183일 이상의 거소(居所)를 둔 개인을 말한다.\n\n2. “비거주자”란 거주자가 아닌 개인을 말한다.\n\n3. “내국법인”이란 「법인세법」 제2조제1호에 따른 내국법인을 말한다.\n\n4. “외국법인”이란 「법인세법」 제2조제3호에 따른 외국법인을 말한다.\n\n5. “사업자”란 사업소득이 있는 거주자를 말한다.\n\n② 제1항에 따른 주소ㆍ거소와 거주자ㆍ비거주자의 구분은 대통령령으로 정한다.\n\n[본조신설 2009. 12. 31.]\n\n\n\n제2조(납세의무) ① 다음 각 호의 어느 하나에 해당하는 개인은 이 법에 따라 각자의 소득에 대한 소득세를 납부할 의무를 진다.\n\n1. 거주자\n\n2. 비거주자로서 국내원천소득(國內源泉所得)이 있는

In [4]:
# 쪼갠 문서를 Embedding 해주기
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

# 환경변수를 불러옴
load_dotenv()

# OpenAI에서 제공하는 Embedding Model을 활용해서 `chunk`를 vector화
embedding = OpenAIEmbeddings(model='text-embedding-3-large')

# openai embedding 모델은 vector 길이가 3072

In [5]:
# 쪼개놓은 document_list를 Embedding을 활용해서 VectorDB에 저장
from langchain_chroma import Chroma

# 데이터를 처음 저장할 때 
database = Chroma.from_documents(documents=document_list, embedding=embedding, collection_name='chroma-tax', persist_directory="./chroma")

# 이미 저장된 데이터를 사용할 때 
# 이미 만들어놓은걸 쓸 때
# database = Chroma(collection_name='chroma-tax', persist_directory="./chroma", embedding_function=embedding)

# Chroma는 기본적으로 in memory라서 파일을 끄면 다 날아감 -> persist_directory 설정
# collection_name은 일반 rdb의 테이블 네임에 해당


# 아래는 Chroma 대신 Pinecone 쓸 때
# import os

# from pinecone import Pinecone
# from langchain_pinecone import PineconeVectorStore

# index_name = 'tax-index'
# pinecone_api_key = os.environ.get("PINECONE_API_KEY")
# pc = Pinecone(api_key=pinecone_api_key)


# database = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)

# 3. 답변 생성을 위한 Retrieval

- `Chroma`에 저장한 데이터를 유사도 검색(`similarity_search()`)를 활용해서 가져옴

In [6]:
database

In [7]:
# 위에서 만든 DataDB에 query를 날릴 수 있음(검색)
# DB를 생성할 때 embedding을 넣었음. 이 embedding을 활용해서 알아서 뒷단에서 유사도 검색을해서 답변을 가져옴
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'

# `k` 값을 조절해서 얼마나 많은 데이터를 불러올지 결정
retrieved_docs = database.similarity_search(query, k=3) # similarity_search()는 유사도검색

In [8]:
retrieved_docs

[Document(id='53946762-a1cb-4dc1-90fb-104090c274bd', metadata={'source': './tax_docs/tax_with_table.docx'}, page_content='나. 그 밖의 배당소득에 대해서는 100분의 14\n\n3. 원천징수대상 사업소득에 대해서는 100분의 3. 다만, 외국인 직업운동가가 한국표준산업분류에 따른 스포츠 클럽 운영업 중 프로스포츠구단과의 계약(계약기간이 3년 이하인 경우로 한정한다)에 따라 용역을 제공하고 받는 소득에 대해서는 100분의 20으로 한다.\n\n4. 근로소득에 대해서는 기본세율. 다만, 일용근로자의 근로소득에 대해서는 100분의 6으로 한다.\n\n5. 공적연금소득에 대해서는 기본세율\n\n5의2.제20조의3제1항제2호나목 및 다목에 따른 연금계좌 납입액이나 운용실적에 따라 증가된 금액을 연금수령한 연금소득에 대해서는 다음 각 목의 구분에 따른 세율. 이 경우 각 목의 요건을 동시에 충족하는 때에는 낮은 세율을 적용한다.\n\n가. 연금소득자의 나이에 따른 다음의 세율\n\n\n\n나. 삭제<2014. 12. 23.>\n\n다. 사망할 때까지 연금수령하는 대통령령으로 정하는 종신계약에 따라 받는 연금소득에 대해서는 100분의 4\n\n5의3. 제20조의3제1항제2호가목에 따라 퇴직소득을 연금수령하는 연금소득에 대해서는 다음 각 목의 구분에 따른 세율. 이 경우 연금 실제 수령연차 및 연금외수령 원천징수세율의 구체적인 내용은 대통령령으로 정한다.\n\n가. 연금 실제 수령연차가 10년 이하인 경우: 연금외수령 원천징수세율의 100분의 70\n\n나. 연금 실제 수령연차가 10년을 초과하는 경우: 연금외수령 원천징수세율의 100분의 60\n\n6. 기타소득에 대해서는 다음에 규정하는 세율. 다만, 제8호를 적용받는 경우는 제외한다.\n\n가. 제14조제3항제8호라목 및 마목에 해당하는 소득금액이 3억원을 초과하는 경우 그 초과하는 분에 대해서는 100분의 30\n\n나. 

# 4. Augmentation을 위한 Prompt 활용

- Retrieval된 데이터는 LangChain에서 제공하는 프롬프트(`"rlm/rag-prompt"`) 사용

In [9]:
# 문서를 가져왔으니 이제 LLM에 질의를 함
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o-mini')

In [10]:
# 문서도 같이 줘야하므로 prompt를 짜야함

prompt = f"""[Identity]
 - 당신은 최고의 한국 소득세 전문가입니다
 - [Context]를 참고해서 사용자의 질문에 답변해주세요

 [Context]
 {retrieved_docs}

 [Question]
 {query}
"""

In [11]:
ai_message = llm.invoke(prompt)

In [12]:
ai_message.content

'연봉 5천만 원인 직장인의 소득세를 계산하기 위해서는 기본세율이 적용됩니다. 한국의 소득세는 누진세 구조로 되어 있기 때문에, 연봉에 따라 세율이 다르게 적용됩니다.\n\n가장 먼저, 연봉 5천만 원의 경우, 2023년 기준으로 소득세를 계산하기 위한 과세표준 구간에 해당하는 세율은 다음과 같습니다.\n\n1. 2천만 원 이하: 6%\n2. 2천만 원 초과 4천만 원 이하: 15%\n3. 4천만 원 초과 8천만 원 이하: 24%\n\n이제 연봉 5천만 원에 대한 세액을 계산해보겠습니다.\n\n1. **첫 2천만 원에 대한 세액**:  \n\\[ 2,000,000 \\times 0.06 = 120,000 \\text{ 원} \\]\n\n2. **다음 2천만 원에 대한 세액**:  \n\\[ 2,000,000 \\times 0.15 = 300,000 \\text{ 원} \\]\n\n3. **4천만 원 초과 5천만 원에 대한 세액**:  \n\\[ 1,000,000 \\times 0.24 = 240,000 \\text{ 원} \\]\n\n위의 세액을 모두 합산하면, 총 소득세는 다음과 같습니다.  \n\\[ 120,000 + 300,000 + 240,000 = 660,000 \\text{ 원} \\]\n\n따라서, 연봉 5천만 원인 직장인의 소득세는 약 660,000원입니다. 이는 세액공제나 기타 사항을 반영하지 않은 단순 계산입니다. 실제 소득세는 다양한 공제 항목에 따라 달라질 수 있습니다.'

'연봉 5천만원인 직장인의 소득세를 계산하기 위해서는 기본세율을 적용해야 합니다. \n\n2023년 현재 한국 소득세의 기본세율은 다음과 같습니다:\n\n- 1,200만원 이하: 6%\n- 1,200만원 초과 ~ 4,600만원 이하: 15%\n- 4,600만원 초과 ~ 8,800만원 이하: 24%\n\n연봉 5천만원은 1,200만원을 초과하고 4,600만원에 해당하므로, 세금 계산은 다음과 같이 진행됩니다:\n\n1. 1,200만원에 대한 세금: 1,200만원 × 6% = 72만원\n2. 1,200만원 초과 부분 (4,600만원까지)에 대한 세금: (4,600만원 - 1,200만원) × 15% = 3,400만원 × 15% = 510만원\n\n따라서 소득세 총액은 다음과 같습니다:\n\n- 총 세금 = 72만원 + 510만원 = 582만원\n\n그러므로 연봉 5천만원인 직장인의 소득세는 약 582만원입니다. \n\n세액 공제나 다른 추가 요인에 따라 최종 세액이 달라질 수 있으니 참고하시기 바랍니다.'

# 5. 답변 생성

- [RetrievalQA](https://docs.smith.langchain.com/old/cookbook/hub-examples/retrieval-qa-chain)를 통해 LLM에 전달
    - `RetrievalQA`는 [create_retrieval_chain](https://python.langchain.com/v0.2/docs/how_to/qa_sources/#using-create_retrieval_chain)으로 대체됨
    - 실제 ChatBot 구현 시 `create_retrieval_chain`으로 변경하는 과정을 볼 수 있음

In [13]:
# langchain은 chain이므로 연결이 됨
# Retrieval를 좀 더 효과적으로 할 수 있음
# RetrievalQAChain 이란걸 쓸 것임 -> Retrieval한 다음에 QA할거다 라는 뜻
# 제일 큰 이점: 프롬프트를 많이 줌

%pip install -U langchain langchainhub --quiet


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [14]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")
prompt

/Users/kyutae/.pyenv/versions/3.11.9/envs/RAG_p311/lib/python3.11/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

### prompt를 자세히보면 페르소나도 주고 retrived context를 사용해서 답변하란 내용도 포함돼있음

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])


In [16]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm, 
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)
# as_retriever는 chroma에만 있는게 아니고
# pinecone, azure 등 다양한 vector database에 다 쓸 수 있음

In [17]:
retriever=database.as_retriever()
retriever.invoke(query)

[Document(id='53946762-a1cb-4dc1-90fb-104090c274bd', metadata={'source': './tax_docs/tax_with_table.docx'}, page_content='나. 그 밖의 배당소득에 대해서는 100분의 14\n\n3. 원천징수대상 사업소득에 대해서는 100분의 3. 다만, 외국인 직업운동가가 한국표준산업분류에 따른 스포츠 클럽 운영업 중 프로스포츠구단과의 계약(계약기간이 3년 이하인 경우로 한정한다)에 따라 용역을 제공하고 받는 소득에 대해서는 100분의 20으로 한다.\n\n4. 근로소득에 대해서는 기본세율. 다만, 일용근로자의 근로소득에 대해서는 100분의 6으로 한다.\n\n5. 공적연금소득에 대해서는 기본세율\n\n5의2.제20조의3제1항제2호나목 및 다목에 따른 연금계좌 납입액이나 운용실적에 따라 증가된 금액을 연금수령한 연금소득에 대해서는 다음 각 목의 구분에 따른 세율. 이 경우 각 목의 요건을 동시에 충족하는 때에는 낮은 세율을 적용한다.\n\n가. 연금소득자의 나이에 따른 다음의 세율\n\n\n\n나. 삭제<2014. 12. 23.>\n\n다. 사망할 때까지 연금수령하는 대통령령으로 정하는 종신계약에 따라 받는 연금소득에 대해서는 100분의 4\n\n5의3. 제20조의3제1항제2호가목에 따라 퇴직소득을 연금수령하는 연금소득에 대해서는 다음 각 목의 구분에 따른 세율. 이 경우 연금 실제 수령연차 및 연금외수령 원천징수세율의 구체적인 내용은 대통령령으로 정한다.\n\n가. 연금 실제 수령연차가 10년 이하인 경우: 연금외수령 원천징수세율의 100분의 70\n\n나. 연금 실제 수령연차가 10년을 초과하는 경우: 연금외수령 원천징수세율의 100분의 60\n\n6. 기타소득에 대해서는 다음에 규정하는 세율. 다만, 제8호를 적용받는 경우는 제외한다.\n\n가. 제14조제3항제8호라목 및 마목에 해당하는 소득금액이 3억원을 초과하는 경우 그 초과하는 분에 대해서는 100분의 30\n\n나. 

In [18]:
ai_message = qa_chain({"query": query})
ai_message

/var/folders/60/zbd68vt105n9svnq858jj8dc0000gn/T/ipykernel_34451/2125453911.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  ai_message = qa_chain({"query": query})


{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '연봉 5천만원인 직장인의 소득세는 해당하는 세율을 적용하여 계산해야 합니다. 근로소득에 대해서는 기본세율이 적용되며, 구체적인 세율 정보는 제공된 문맥에 포함되어 있지 않습니다. 따라서 정확한 소득세 금액은 알 수 없습니다.'}

In [19]:
# 기존 방식
# 데이터 생성 -> docs를 가져옴(retrieved_docs) -> prompt를 짜서 -> llm에 invoke

# qa chain을 사용하면
# 뒷단에 알아서 데이터 들어가고(retriever=database.as_retriever())
# 우리는 query만 주면 됨 

# 6. Retrieval을 위한 keyword 사전 활용

- Knowledge Base에서 사용되는 keyword를 활용하여 사용자 질문 수정
- LangChain Expression Language (LCEL)을 활용한 Chain 연계

In [20]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

dictionary = ["사람을 나타내는 표현 -> 거주자"]

prompt = ChatPromptTemplate.from_template(f"""
    사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
    만약 변경할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 않아도 됩니다.
    그런 경우에는 질문만 리턴해주세요
    사전: {dictionary}
    
    질문: {{question}}
""")

dictionary_chain = prompt | llm | StrOutputParser()
tax_chain = {"query": dictionary_chain} | qa_chain

In [21]:
new_question = dictionary_chain.invoke({"question": query})

In [22]:
print(query)
print(new_question)

연봉 5천만원인 직장인의 소득세는 얼마인가요?
연봉 5천만원인 거주자의 소득세는 얼마인가요?


In [23]:
ai_response = tax_chain.invoke({"question": query})

In [24]:
ai_response

{'query': '연봉 5천만원인 거주자의 소득세는 얼마인가요?',
 'result': '연봉 5천만원인 거주자의 소득세는 84만원 + (1,400만원을 초과하는 금액의 15%)로 계산됩니다. 즉, 5천만원에서 1천400만원을 빼고 그 차액에 15%를 곱한 금액을 더하면 됩니다. 최종적으로 소득세는 84만원 + (3,600,000 원 * 0.15) = 84만원 + 540,000원 = 624,000원이 됩니다.'}